In [12]:
!pip install sec-api
!pip install requests


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [38]:
import os
import warnings
import csv
import requests
import pandas as pd
import json

warnings.filterwarnings("ignore")
dir_path = os.getcwd()
parent_directory = os.path.dirname(dir_path)
os.environ["ROOT_DIR"] = parent_directory
print(dir_path)
print(parent_directory)

/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss/data_collection
/Users/rouzbeh.farahmand/PycharmProjects/commit/financial-vss


In [41]:
api_key = '05b0ca0aee222c6fc85051a9725a9c5b8b3cedd76ff67344de14e0b242a26fd4'
from sec_api import QueryApi
queryApi = QueryApi(api_key=api_key)

In [42]:
query = {
  "query": { "query_string": { 
      "query": "formType:\"10-K\" AND ticker:TSLA", # only 10-Ks
  }},
  "from": "0", # start returning matches from position null, i.e. the first matching filing 
  "size": "1"  # return just one filing
}

response = queryApi.get_filings(query)

In [43]:
response

{'total': {'value': 19, 'relation': 'eq'},
 'query': {'from': 0, 'size': 1},
 'filings': [{'id': '72ccf726af50ba95c163db42643b93ab',
   'accessionNo': '0001628280-24-002390',
   'cik': '1318605',
   'ticker': 'TSLA',
   'companyName': 'Tesla, Inc.',
   'companyNameLong': 'Tesla, Inc. (Filer)',
   'formType': '10-K',
   'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
   'filedAt': '2024-01-26T21:00:20-05:00',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/0001628280-24-002390.txt',
   'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/0001628280-24-002390-index.htm',
   'linkToXbrl': '',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm',
   'entities': [{'companyName': 'Tesla, Inc. (Filer)',
     'cik': '1318605',
     'irsNo': '912197729',
     'stateOfIncorporation': 'DE',
     'fiscalYearEnd': '1231',
     'type': '10-

In [17]:
def download_sample_russell_3000_data():
    url = 'https://www.ishares.com/us/products/239714/ishares-russell-3000-etf/1467271812596.ajax?fileType=csv&fileName=IWV_holdings&dataType=fund&asOfDate=20221230'
    response = requests.get(url)
    file_name = f'{parent_directory}/resources/russell-3000.csv'
    with open(file_name, 'wb') as f:
        f.write(response.content)
        
    with open(file_name, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = list(reader)
    
    empty_row_indicies = [i for i in range(len(rows)) if (len(rows[i]) == 0 or '\xa0' in rows[i])]
    
    print('Empty rows:', empty_row_indicies)
    
    start = empty_row_indicies[0] + 1
    end = empty_row_indicies[1]
    cleaned_rows = rows[start:end]
    
    with open(file_name, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(cleaned_rows)
    return file_name

In [45]:
russel_file_name  = f'{parent_directory}/resources/russell-3000.csv' #download_sample_russell_3000_data()

In [46]:
holdings = pd.read_csv(russel_file_name)
holdings.head(10)

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date
0,AAPL,APPLE INC,Information Technology,Equity,"559,365,151.11",5.16,"559,365,151.11","4,305,127.00",129.93,United States,NASDAQ,USD,1.0,USD,-
1,MSFT,MICROSOFT CORP,Information Technology,Equity,"513,917,712.42",4.74,"513,917,712.42","2,142,931.00",239.82,United States,NASDAQ,USD,1.0,USD,-
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,"213,823,596.00",1.97,"213,823,596.00","2,545,519.00",84.00,United States,NASDAQ,USD,1.0,USD,-
3,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Financials,Equity,"159,603,687.60",1.47,"159,603,687.60","516,684.00",308.90,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
4,GOOGL,ALPHABET INC CLASS A,Communication,Equity,"151,996,026.75",1.40,"151,996,026.75","1,722,725.00",88.23,United States,NASDAQ,USD,1.0,USD,-
5,UNH,UNITEDHEALTH GROUP INC,Health Care,Equity,"142,028,859.84",1.31,"142,028,859.84","267,888.00",530.18,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
6,GOOG,ALPHABET INC CLASS C,Communication,Equity,"135,557,878.61",1.25,"135,557,878.61","1,527,757.00",88.73,United States,NASDAQ,USD,1.0,USD,-
7,JNJ,JOHNSON & JOHNSON,Health Care,Equity,"133,147,817.70",1.23,"133,147,817.70","753,738.00",176.65,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
8,XOM,EXXON MOBIL CORP,Energy,Equity,"130,179,148.40",1.20,"130,179,148.40","1,180,228.00",110.30,United States,New York Stock Exchange Inc.,USD,1.0,USD,-
9,JPM,JPMORGAN CHASE & CO,Financials,Equity,"112,022,178.30",1.03,"112,022,178.30","835,363.00",134.10,United States,New York Stock Exchange Inc.,USD,1.0,USD,-


In [47]:
# create batches of tickers: [[A,B,C], [D,E,F], ...]
# a single batch has a maximum of max_length_of_batch tickers
def create_batches(tickers = [], max_length_of_batch = 100):
  batches = [[]]

  for ticker in tickers:
    if len(batches[len(batches)-1]) == max_length_of_batch:
      batches.append([])

    batches[len(batches)-1].append(ticker)

  return batches


batches = create_batches(list(holdings['Ticker']))


In [48]:
batches

[['AAPL',
  'MSFT',
  'AMZN',
  'BRKB',
  'GOOGL',
  'UNH',
  'GOOG',
  'JNJ',
  'XOM',
  'JPM',
  'PG',
  'NVDA',
  'CVX',
  'V',
  'HD',
  'TSLA',
  'LLY',
  'MA',
  'PFE',
  'ABBV',
  'MRK',
  'META',
  'PEP',
  'KO',
  'BAC',
  'AVGO',
  'TMO',
  'COST',
  'WMT',
  'CSCO',
  'MCD',
  'ABT',
  'DHR',
  'ACN',
  'NEE',
  'VZ',
  'LIN',
  'DIS',
  'PM',
  'WFC',
  'ADBE',
  'BMY',
  'CMCSA',
  'TXN',
  'RTX',
  'COP',
  'HON',
  'NKE',
  'AMGN',
  'T',
  'NFLX',
  'UPS',
  'CRM',
  'UNP',
  'IBM',
  'SCHW',
  'ORCL',
  'CAT',
  'LOW',
  'ELV',
  'QCOM',
  'CVS',
  'DE',
  'SBUX',
  'LMT',
  'GS',
  'SPGI',
  'INTC',
  'GILD',
  'INTU',
  'BLK',
  'MS',
  'AMD',
  'BA',
  'MDT',
  'PLD',
  'ADP',
  'CI',
  'AMT',
  'ISRG',
  'TJX',
  'CB',
  'GE',
  'MDLZ',
  'AXP',
  'C',
  'SYK',
  'ADI',
  'AMAT',
  'TMUS',
  'MO',
  'MMC',
  'PYPL',
  'NOC',
  'DUK',
  'BKNG',
  'NOW',
  'SO',
  'PGR',
  'EOG'],
 ['SLB',
  'REGN',
  'VRTX',
  'BDX',
  'ZTS',
  'TGT',
  'APD',
  'ITW',
  'BSX',
  'M

In [49]:
from sec_api import QueryApi, RenderApi
from pathlib import Path
import multiprocessing


def download_10K_metadata(tickers = [], start_year = 2020, end_year = 2023):
  if Path('metadata.csv').is_file():
    result = pd.read_csv('metadata.csv')
    return result

  print('✅ Starting download process')

  # create ticker batches, with 100 tickers per batch
  batches = create_batches(tickers)
  frames = []

  for year in range(start_year, end_year + 1):
    for batch in batches:
      tickers_joined = ', '.join(batch)
      ticker_query = 'ticker:({})'.format(tickers_joined)

      query_string = '{ticker_query} AND filedAt:[{start_year}-01-01 TO {end_year}-12-31] AND formType:"10-K" AND NOT formType:"10-K/A" AND NOT formType:NT'.format(ticker_query=ticker_query, start_year=year, end_year=year)

      query = {
        "query": { "query_string": { 
            "query": query_string,
            "time_zone": "America/New_York"
        } },
        "from": "0",
        "size": "200",
        "sort": [{ "filedAt": { "order": "desc" } }]
      }

      response = queryApi.get_filings(query)

      filings = response['filings']

      metadata = list(map(lambda f: {'ticker': f['ticker'], 
                                     'cik': f['cik'], 
                                     'formType': f['formType'], 
                                     'filedAt': f['filedAt'], 
                                     'filingUrl': f['linkToFilingDetails']}, filings))

      df = pd.DataFrame.from_records(metadata)

      frames.append(df)

    print('✅ Downloaded metadata for year', year)


  result = pd.concat(frames)
  result.to_csv(f'{parent_directory}/resources/metadata.csv', index=False)

  number_metadata_downloaded = len(result)
  print('✅ Downloaded completed. Metadata downloaded for {} filings.'.format(number_metadata_downloaded))

  return result


tickers = list(holdings['Ticker'])

metadata = download_10K_metadata(tickers=tickers, start_year=2021, end_year=2023)

✅ Starting download process


KeyboardInterrupt: 

In [23]:
import os

renderApi = RenderApi(api_key=api_key)

def download_filing(metadata):
  try:
    ticker = metadata['ticker']
    new_folder = f"{parent_directory}/resources/filings/" + ticker

    if not os.path.isdir(new_folder):
      os.makedirs(new_folder)

    url = metadata['filingUrl'].replace('ix?doc=/', '')
    file_content = renderApi.get_filing(url)
    file_name = url.split("/")[-1] 

    with open(new_folder + "/" + file_name, "w") as f:
      f.write(file_content)
  except:
     print('❌ {ticker}: downloaded failed: {url}'.format(ticker=ticker, url=url))

In [24]:
# downloaded sample filing
download_filing(metadata.iloc[0])
print('✅ Sample 10-K filing downloaded for {}'.format(metadata.iloc[0]['ticker']))

✅ Sample 10-K filing downloaded for AVGO


In [52]:
metadata = pd.read_csv(f"{parent_directory}/resources/metadata.csv")

In [53]:
metadata.head(2)

,ticker,cik,formType,filedAt,filingUrl
0,AVGO,1730168,10-K,2021-12-17T16:42:51-05:00,https://www.sec.gov/Archives/edgar/data/173016...
1,AMAT,6951,10-K,2021-12-17T16:14:51-05:00,https://www.sec.gov/Archives/edgar/data/6951/0...


In [56]:

READER_API_ENDPOINT = "https://api.sec-api.io/filing-reader"
def get_pdf_filings(metadata, tickers):
    print("download")
    for ticker in tickers:
        try:
            filing_urls_date = metadata[metadata['ticker'] == ticker][['filingUrl','filedAt']].values
            company_df = holdings[holdings['Ticker'] == ticker]
            company_json = json.loads(company_df.to_json(orient='records'))
            new_folder = f"{parent_directory}/resources/filings/" + ticker
            
            if not os.path.isdir(new_folder):
                os.makedirs(new_folder)
                
            
            if len(company_json) > 0 :
                new_company_json_file = f"{new_folder}/{ticker}-metadata.json"
                # Write to the file
                with open(new_company_json_file, 'w') as json_file:
                    json.dump(company_json[0], json_file, indent=4)
            
            for filing_url_date in filing_urls_date:
                filing_url = str(filing_url_date[0])
                filing_date = str(filing_url_date[1]).split('-')[0]
                api_url = READER_API_ENDPOINT + "?token=" + api_key + "&url=" + filing_url + "&type=pdf"
                print(api_url)
                response = requests.get(api_url)
                
            
                new_file_name = f"{new_folder}/{ticker}-{filing_date}-10K.pdf"
                with open(new_file_name, 'wb') as f:
                    f.write(response.content)
                
                with open(new_file_name, 'wb') as f:
                    f.write(response.content)
                print(f'✅ Sample 10-K filing downloaded for {ticker}-{filing_date}')
            
        except Exception as ex:
            print(f'❌ {ticker}: downloaded failed for ticker {ticker} due to {str(ex)}')
            
    print(f'✅ Finished download process for {len(tickers)} tickers')
    

In [57]:
get_pdf_filings(metadata, tickers)

download
https://api.sec-api.io/filing-reader?token=05b0ca0aee222c6fc85051a9725a9c5b8b3cedd76ff67344de14e0b242a26fd4&url=https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/aapl-20210925.htm&type=pdf
✅ Sample 10-K filing downloaded for AAPL-2021
https://api.sec-api.io/filing-reader?token=05b0ca0aee222c6fc85051a9725a9c5b8b3cedd76ff67344de14e0b242a26fd4&url=https://www.sec.gov/Archives/edgar/data/320193/000032019322000108/aapl-20220924.htm&type=pdf
✅ Sample 10-K filing downloaded for AAPL-2022
https://api.sec-api.io/filing-reader?token=05b0ca0aee222c6fc85051a9725a9c5b8b3cedd76ff67344de14e0b242a26fd4&url=https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm&type=pdf
✅ Sample 10-K filing downloaded for AAPL-2023
https://api.sec-api.io/filing-reader?token=05b0ca0aee222c6fc85051a9725a9c5b8b3cedd76ff67344de14e0b242a26fd4&url=https://www.sec.gov/Archives/edgar/data/789019/000156459021039151/msft-10k_20210630.htm&type=pdf
✅ Sample 10-K filing download

KeyboardInterrupt: 